<a href="https://colab.research.google.com/github/mariam2002212/JS-simplePbls/blob/main/ml2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

load dataset

In [30]:
import pandas as pd
data = pd.read_csv('weather_forecast_data.csv')
print(data.head())

   Temperature   Humidity  Wind_Speed  Cloud_Cover     Pressure     Rain
0    19.096119  71.651723   14.782324    48.699257   987.954760  no rain
1    27.112464  84.183705   13.289986    10.375646  1035.430870  no rain
2    20.433329  42.290424    7.216295     6.673307  1033.628086  no rain
3    19.576659  40.679280    4.568833    55.026758  1038.832300  no rain
4    19.828060  93.353211    0.104489    30.687566  1009.423717  no rain


preprocessing

first we want to identify the missing data

In [31]:
missing_data = data.isnull().sum()
print("Missing values in each column:")
print(missing_data)

Missing values in each column:
Temperature    25
Humidity       40
Wind_Speed     32
Cloud_Cover    33
Pressure       27
Rain            0
dtype: int64


then we want to handle the missing data
first, technique--> dropping missing values

In [19]:
dropped_data = data.dropna()
print("Data after dropping missing values:")
print(dropped_data.head())

Data after dropping missing values:
   Temperature   Humidity  Wind_Speed  Cloud_Cover     Pressure     Rain
0    19.096119  71.651723   14.782324    48.699257   987.954760  no rain
1    27.112464  84.183705   13.289986    10.375646  1035.430870  no rain
2    20.433329  42.290424    7.216295     6.673307  1033.628086  no rain
3    19.576659  40.679280    4.568833    55.026758  1038.832300  no rain
4    19.828060  93.353211    0.104489    30.687566  1009.423717  no rain


second technique, replacing them with the average of the feature."handled both numerical features and non numerical features"

1st, Separate numeric and non-numeric features

In [20]:
numeric_features = data.select_dtypes(include=['number']).columns
non_numeric_features = data.select_dtypes(exclude=['number']).columns

2nd, calc avg value for numeric features

In [21]:
data_replaced = data.copy()
data_replaced[numeric_features] = data_replaced[numeric_features].fillna(data_replaced[numeric_features].mean())

3rd, handle non numeric features "using mode, is used for categorical data in our case"


In [22]:
for feature in non_numeric_features:
    data_replaced[feature] = data_replaced[feature].fillna(data_replaced[feature].mode()[0])

In [23]:
print("\nData after replacing missing values with the average of the feature:")
print(data_replaced.head())


Data after replacing missing values with the average of the feature:
   Temperature   Humidity  Wind_Speed  Cloud_Cover     Pressure     Rain
0    19.096119  71.651723   14.782324    48.699257   987.954760  no rain
1    27.112464  84.183705   13.289986    10.375646  1035.430870  no rain
2    20.433329  42.290424    7.216295     6.673307  1033.628086  no rain
3    19.576659  40.679280    4.568833    55.026758  1038.832300  no rain
4    19.828060  93.353211    0.104489    30.687566  1009.423717  no rain


Splitting our data to training and testing for training and evaluating our
models

train/test split "on non handled data"

In [ ]:
from sklearn.model_selection import train_test_split

features = data.iloc[:, :-1]
target = data.iloc[:, -1] #akher column hwa eltarget w elba2y features

feature_train, feature_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=20)
print("\nTraining set size:", feature_train.shape)
print("Testing set size:", feature_test.shape)

train/test split"on the handeled data with the dropping missing data technique"

In [75]:
from sklearn.model_selection import train_test_split

features = dropped_data.iloc[:, :-1]
target = dropped_data.iloc[:, -1] #akher column hwa eltarget w elba2y features

feature_train, feature_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=24)
print("\nTraining set size:", feature_train.shape)
print("Testing set size:", feature_test.shape)


Training set size: (1877, 5)
Testing set size: (470, 5)


train/test split"on the handeled data with the replacing missing data technique"

In [38]:
from sklearn.model_selection import train_test_split

features = data_replaced.iloc[:, :-1]
target = data_replaced.iloc[:, -1] #akher column hwa eltarget w elba2y features

feature_train, feature_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=20)
print("\nTraining set size:", feature_train.shape)
print("Testing set size:", feature_test.shape)


Training set size: (2000, 5)
Testing set size: (500, 5)


feature scaling

Scale the numeric features in the training set

In [60]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
feature_train_numeric = feature_train[numeric_features]
feature_train_scaled = scaler.fit_transform(feature_train_numeric)

feature_train[numeric_features] = feature_train_scaled

print("Scaled Training Data:")
print(feature_train.head())

Scaled Training Data:
      Temperature  Humidity  Wind_Speed  Cloud_Cover  Pressure
1393     0.316341  0.953360   -0.773017    -0.293621 -0.844798
44      -0.882151 -0.800889   -1.348096    -1.429369 -1.202206
1087     0.717007  0.128807    0.437261     1.432991 -1.148291
1268     0.400735 -1.680496   -0.577251     0.048602  1.221948
1801     1.002973  0.843221    1.197734     1.517268  0.749780


Scale the numeric features in the testing set

In [61]:
feature_test_numeric = feature_test[numeric_features]
feature_test_scaled = scaler.transform(feature_test_numeric)

feature_test[numeric_features] = feature_test_scaled

print("Scaled Testing Data:")
print(feature_test.head())

Scaled Testing Data:
      Temperature  Humidity  Wind_Speed  Cloud_Cover  Pressure
689     -0.715958 -0.428289   -1.670461     1.489984 -0.658674
1037     0.128353  0.014122   -0.647950     1.466338 -0.472686
1422    -1.489486 -1.145145    0.982570     1.054588 -0.937307
1557    -0.152608  0.679646    1.497272    -0.374085  0.413014
1419    -0.672651  0.024019   -1.530189     1.537018  1.442861


implement decision tree

In [76]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

decision_tree_model = DecisionTreeClassifier(random_state = 20)

decision_tree_model.fit(feature_train, target_train)#train el model
predicted_target = decision_tree_model.predict(feature_test)

accuracy = accuracy_score(target_test, predicted_target)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9957
